Since we are going to use what cffi calls the "API-level, out-of-line", we need to define a build script (`build_rmath.py`) that we will use to compile the Rmath source and produce an importable extension module. The notebook "cell magic", `%%file` will write the contents of the below cell to a file.

In [19]:
%%file build_rmath.py

import glob
import os
import platform

from cffi import FFI


include_dirs = [os.path.join('externals', 'Rmath-julia', 'src'),
                os.path.join('externals', 'Rmath-julia', 'include')]

rmath_src = glob.glob(os.path.join('externals', 'Rmath-julia', 'src', '*.c'))

# Take out dSFMT dependant files; Just use the basic rng
rmath_src = [f for f in rmath_src if ('librandom.c' not in f) and ('randmtzig.c' not in f)]

extra_compile_args = ['-DMATHLIB_STANDALONE']
if platform.system() == 'Windows':
    extra_compile_args.append('-std=c99')

ffi = FFI()
ffi.set_source('_rmath_ffi', '#include <Rmath.h>',
        include_dirs=include_dirs,
        sources=rmath_src,
        libraries=[],
        extra_compile_args=extra_compile_args)

# This is an incomplete list of the available functions in Rmath
# but these are sufficient for our example purposes
ffi.cdef('''\
double pnorm(double, double, double, int, int);
double qnorm(double, double, double, int, int);
double runif(double, double);
void set_seed(unsigned int, unsigned int);
void get_seed(unsigned int *, unsigned int *);
''')

if __name__ == '__main__':
    # Normally set verbose to `True`, but silence output
    # for reduced notebook noise
    ffi.compile(verbose=False)

Overwriting build_rmath.py


Then we simply run the script as below assuming we have a properly configured C compiler on our system. For larger projects [integration with setuptools is supported](http://cffi.readthedocs.org/en/release-1.5/cdef.html#preparing-and-distributing-modules). The exclamation point tells the notebook to execute the following command in a system shell.

In [20]:
!python build_rmath.py

We should now have an extension module named `_rmath_ffi` that gives us access to the functions whose prototypes we enumerated in the `ffi.cdef(...)`.

In [10]:
import _rmath_ffi